# <p style="background-color: #f5df18; padding: 10px;"> Foundations of Astronomical Data Science | **Photometry** </p>



### <strong>Instructor: <span style="color: darkblue;">Name (Affliation)</span></strong>

Estimated completion time: 🕚 40 minutes



<div style="display: flex;">
    <div style="flex: 1; margin-right: 100px;">
        <h2>Questions</h2>
        <ul>
            <li>How do we use Matplotlib to define a polygon and select points that fall inside it?</li>
        </ul>
    </div>
    <div style="flex: 1;">
        <h2>Learning Objectives</h2>
        <ul>
            <li>Use isochrone data to specify a polygon and determine which points fall inside it.</li>
            <li>Use Matplotlib features to customize the appearance of figures.</li>
        </ul>
    </div>
</div>


In the previous episode we downloaded photometry data from Pan-STARRS,
which is available from the same server we have been using to get Gaia
data.

The next step in the analysis is to select candidate stars based on
the photometry data.
The following figure from the paper is a color-magnitude diagram
showing the stars we previously selected based on proper motion:


<div>
<img src="https://datacarpentry.org/astronomy-python/fig/gd1-3.png" width="800"/>
</div>

In orange is a [theoretical isochrone](https://en.wikipedia.org/wiki/Stellar_isochrone), showing where we expect the stars
in GD-1 to fall based on the metallicity and age of their original
globular cluster.

By selecting stars in the shaded area, we can further distinguish the
main sequence of GD-1 from mostly younger background stars.


# Outline
---

1. We will reload the data from the previous episode and make a
  color-magnitude diagram.

2. We will use an isochrone computed by MIST to specify a polygonal
  region in the color-magnitude diagram and select the stars inside of it.

# Plotting photometry data
---

Now that we have photometry data from Pan-STARRS, we can produce a
[color-magnitude
diagram](https://coolwiki.ipac.caltech.edu/index.php/Color-Magnitude_and_Color-Color_plots_Overview)
to replicate the diagram from the original paper:

<div>
<img src="https://datacarpentry.org/astronomy-python/fig/gd1-3.png" width="800"/>
</div>

The y-axis shows the apparent magnitude of each source with the [g
filter](https://en.wikipedia.org/wiki/Photometric_system).

The x-axis shows the difference in apparent magnitude between the g
and i filters, which indicates color.

## The color magnitude diagram of GD-1 and foreground stars

As a pathologist can easily point to tumor on a biopsy slide, so too can astronomers who study stellar populations see two stellar main groups of stars in this color magnitude diagram, one from an old star cluster (presumably, GD-1), and the other, stars much closer, but at every distance between the Earth and the cluster ("foreground"). The color magnitude diagram is a technique developed to separate these features out just as pathologists have techniques to contrast human tissue.
The color of a star is primarily related to the star's surface temperature, with bluer stars indicating higher temperatures and redder stars indicating lower temperatures. This logic is not too far off from the color at the bottom of a match flame compared to the top.


Foreground Stars:
To know the temperature of a star, you first need to know its distance and to account for the dust between us and the star. You can guess the effect of distance. A star farther away will be fainter (lower y-axis value) than the same star closer (think of car headlights as they approach). Dust will remove light from the star's path to our telescopes, which makes the star seem like it has less energy than it otherwise would have, which makes it do two things on this diagram: 1) look fainter (lower on the y-axis; larger magnitude value) and 2) look cooler (higher x-axis value). The stars spread throughout the diagram are all stars bright enough to be detected in our Milky Way between GD-1 and us but made fainter and redder (spread to the lower-right) by their spread in distance from us and the amount dust in the line of sight.


GD-1:
The pile up of stars in the lower-left quadrant of this diagram are interesting because it suggests something is at the same distance with the same amount of dust in the way. When we use our knowledge of theoretical astrophysics (independently calculated outside this work) to estimate how bright a population of old stars would be if it were at the distance of GD-1, we get that solid red line. The exact values of age and metallicity ([Fe/H] value) is a variable needed to reproduce the theoretical isochrone, but frankly, the choice could vary a lot and still would fit the data well.

[More on color-magnitude diagrams and their theoretical counterpart, here.](https://spiff.rit.edu/classes/ladder/lectures/ordinary_stars/ordinary.html)


With the photometry we downloaded from the PanSTARRS table into
`candidate_df` we can now recreate this plot.

In [ ]:
### Recreate color-magnitude diagram ### 






We have assigned the color and magnitude to variables `x` and `y`, respectively.  
We have done this out of convenience and to keep the code readable since the
table variables and column names are long and `x` includes an operation
between two columns.

We can zoom in on the region of interest by setting the range of
x and y values displayed with the `xlim` and `ylim` functions.
If we put the higher value first in the `ylim` call, this will invert
the y-axis, putting fainter magnitudes at the bottom.

In [ ]:
### Zoom in on the region of interest ###





Our figure does not look exactly like the one in the paper because we
are working with a smaller region of the sky, so we have fewer
stars.  But the main sequence of GD-1 appears as an overdense region in the lower left.

We want to be able to make this plot again, with any selection of PanSTARRs photometry,
so this is a natural time to put it into a function that accepts as input
an Astropy `Table` or Pandas `DataFrame`, as long as
it has columns named `g_mean_psf_mag` and `i_mean_psf_mag`. To do this we will change
our variable name from `candidate_df` to the more generic `dataframe`.

In [1]:
### Define function "plot_cmd" to assist in recreating figure ####





Here are the results:

In the next section we will use an isochrone to specify a polygon that contains this overdense region.

# Isochrone
---

Given our understanding of the age, metallicity, and distance to GD-1 we can overlay a
theoretical isochrone for GD-1 from the MESA Isochrones and Stellar Tracks and better identify the main sequence of GD-1.

## 🔔 Calculating Isochrone
---

In fact, we can use [MESA Isochrones \& Stellar Tracks](https://waps.cfa.harvard.edu/MIST/) (MIST)
to compute it for us.
Using the [MIST Version 1.2 web interface](https://waps.cfa.harvard.edu/MIST/interp_isos.html),
we computed an isochrone with the following parameters:

- Rotation initial v/v\_crit = 0.4
- Single age, linear scale = 12e9
- Composition [Fe/H] = -1.35
- Synthetic Photometry, PanStarrs
- Extinction av = 0

# Making a polygon
---

The MIST isochrone files available on the website above can not be directly plotted over our data.
We have selected the relevant part of the isochrone, the filters we are interested in, and scaled the photometry to the distance of GD-1
([details here](https://datacarpentry.org/astronomy-python/instructor/calculating_MIST_isochrone.html)).
Now we can read in the results which you downloaded as part of the [setup instructions](https://datacarpentry.org/astronomy-python/instructor/index.html#setup):

In [ ]:
### Read in 'gd1_isochrone.hdf5' ### 




Here is what the isochrone looks like on the color-magnitude diagram.

In [ ]:
### Overplot isochrone on the color-magnitude diagram ###





In the bottom half of the figure, the isochrone passes through the
overdense region where the stars are likely to belong to GD-1.

Although some stars near the top half of the isochrone likely belong to GD-1,
these represent stars that have evolved off the main sequence. The density of GD-1 stars in this region is therefore
much less and the contamination with other stars much greater. So to get the purest sample of GD-1 stars we will select only stars on the main sequence.

So we will select the part of the isochrone that lies in the overdense region.

`g_mask` is a Boolean Series that is `True` where `g` is between 18.0 and 21.5.

We can use it to select the corresponding rows in `iso_df`:

Now, to select the stars in the overdense region, we have to define a
polygon that includes stars near the isochrone.

Here is our plot with these boundaries:

In [ ]:
### add boundaries to the cmd ###






# Which points are in the polygon?
---

Matplotlib provides a `Polygon` object that we can use to check which
points fall in the polygon we just constructed.

To make a `Polygon`, we need to assemble `g`, `left_color`, and
`right_color` into a loop, so the points in `left_color` are connected
to the points of `right_color` in reverse order.

We will use a "slice index" to reverse the elements of `right_color`.
As explained in the [NumPy
documentation](https://numpy.org/doc/stable/reference/arrays.indexing.html),
a slice index has three parts separated by colons:

- `start`: The index of the element where the slice starts.

- `stop`: The index of the element where the slice ends.

- `step`: The step size between elements.

```
reverse_right_color = right_color[::-1]
```


In this example, `start` and `stop` are omitted, which means all
elements are selected.

And `step` is `-1`, which means the elements are in reverse order.

To combine the `left_color` and `right_color` arrays we will use the NumPy `append` function
which takes two arrays as input, and outputs them combined into a single array.


In [ ]:
### Use the NumPy append function to create a single array ###





We can repeat combine these two lines of code into a single line to create a corresponding loop with the elements of `g` in forward and reverse order.

Here is the loop on our plot:

In [ ]:
### Add the loop to the cmd ###





To make a `Polygon`, it will be useful to put `color_loop` and
`mag_loop` into a `DataFrame`. This is convenient for two reasons - first, `Polygon`
is expecting an Nx2 array and the `DataFrame` provides an easy way for us to pass that
in that is also descriptive for us. Secondly, for reproducibility of our work, we may want
to save the region we use to select stars, and the `DataFrame`, as we have already seen, allows us to save into a variety of formats.

Now we can pass `loop_df` to `Polygon`:

The result is a `Polygon` object which has a `contains_points` method.
This allows us to pass `polygon.contains_points` a list of points and
for each point it will tell us if the point is contained within the polygon.
A point is a tuple with two elements, x and y.


## <p style="background-color: #f5df18; padding: 10px;"> 🛑 Exercise (5 Minutes) </p>
---

When we encounter a new object, it is good to create a toy example to test
that it does what we think it does. Define a list of two points (represented as two tuples),
one that should be inside the polygon and one that should be outside the polygon. Call
`contains_points` on the polygon we just created, passing it the list of points you
defined, to verify that the results are as expected.

In [ ]:
### your answer here ###


We are almost ready to select stars whose photometry data falls in
this polygon.  But first we need to do some data cleaning.


# Save the polygon
---

[Reproducibile
research](https://en.wikipedia.org/wiki/Reproducibility#Reproducible_research)
is "the idea that ... the full computational environment used to
produce the results in the paper such as the code, data, etc. can be
used to reproduce the results and create new work based on the
research."

This lesson is an example of reproducible research because
it contains all of the code needed to reproduce the results, including
the database queries that download the data and the analysis.

In this episode, we used an isochrone to derive a polygon, which we used
to select stars based on photometry.
So it is important to record the polygon as part of the data analysis pipeline.

Here is how we can save it in an HDF5 file.

In [ ]:
### Save loop to HDF5 file ###





# Selecting based on photometry
---

Now we will check how many of the candidate stars are inside the polygon we chose.
`contains_points` expects a list of (x,y) pairs. As with creating the `Polygon`, `DataFrames` are
a convenient way to pass the colors and magnitudes for all of our stars in `candidates_df` to our `Polygon` to see
which candidates are inside the polygon. We will start by putting color and magnitude data from `candidate_df` into a new `DataFrame`.

Which we can pass to `contains_points`:

The result is a Boolean array.

## <p style="background-color: #f5df18; padding: 10px;"> 🛑 Exercise (5 Minutes) </p>
---

Boolean values are stored as 0s and 1s. `FALSE` = 0 and `TRUE` = 1. Use this information
to determine the number of stars that fall inside the polygon.

In [ ]:
### your answer here ####

Now we can use `inside_mask` as a mask to select stars that fall inside the polygon.

We will make a color-magnitude plot one more time, highlighting the selected stars with green markers.

The selected stars are, in fact, inside the polygon,
which means they have photometry data consistent with GD-1.

Finally, we can plot the coordinates of the selected stars:

This example includes the new Matplotlib command `figure`, which creates the larger canvas that the subplots are placed on.  In previous examples, we didn't have
to use this function; the figure was created automatically.  But when
we call it explicitly, we can provide arguments like `figsize`, which
sets the size of the figure. It also returns a figure object which we will
use to further customize our plotting in the next episode.

In the example above we also used TeX markup in our axis labels so that they render as the
Greek letter `$\phi$` with subscripts for `1` and `2`.
Matplotlib also allows us to write basic TeX markup by wrapping the text we want
rendered as TeX with `$` and then using TeX commands inside. This basic rendering
is performed with [mathtext](https://matplotlib.org/stable/tutorials/text/mathtext.html);
more advanced rendering with LaTex can be done with the `usetex` option in `rcParams`
which we will discuss in Episode 7.

In the next episode we are going to make this plot several more times, so it makes sense to
make a function. As we have done with previous functions we can copy and paste what we just wrote,
replacing the specific variable `winner_df` with the more generic `df`.

In [ ]:
def plot_cmd_selection(df):
    x = df['phi1']
    y = df['phi2']

    plt.plot(x, y, 'ko', markersize=0.7, alpha=0.9)

    plt.xlabel('$\phi_1$ [deg]')
    plt.ylabel('$\phi_2$ [deg]')
    plt.title('Proper motion + photometry selection', fontsize='medium')

    plt.axis('equal')

And here is how we use the function.

# 💾 Write the data
---

Finally, we will write the selected stars to a file.


In [ ]:
### save winner_df as an hdf ###





# Summary
---

In this episode, we used photometry data from Pan-STARRS to draw a
color-magnitude diagram.
We used an isochrone to define a polygon and select stars we think are
likely to be in GD-1.  Plotting the results, we have a clearer picture
of GD-1, similar to Figure 1 in the original paper.

# <p style="background-color: #f5df18; padding: 10px;"> 🗝️ Key points</p>
---

- Matplotlib provides operations for working with points, polygons, and other geometric entities, so it is not just for making figures.
- Use Matplotlib options to control the size and aspect ratio of figures to make them easier to interpret.
- Record every element of the data analysis pipeline that would be needed to replicate the results.
